In [1]:
import pandas as pd
import os
import time
import numpy as np
import torch
import gc


In [2]:
import logging
logging.basicConfig(filename="../datax/log.log", format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import seaborn as sns
from scipy import stats
from statistics import NormalDist
import matplotlib.pyplot as plt

In [4]:
torch.__version__

'1.12.1+cu113'

In [5]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cuda', index=0)

In [7]:
torch.cuda.memory_allocated()

0

In [8]:
! nvidia-smi

Sat Apr 22 03:58:27 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA TITAN RTX                On | 00000000:02:00.0 Off |                  N/A |
| 41%   38C    P8               15W / 280W|     20MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Logits Extractor
>
> Extracting Tensor Logits from a given Neural Code Model @xxxxxp
>

In [9]:
def c_eleuther( model_type, returnModel = False, cache_path = "../datax/cache"):
    ''' Eleuther and Salesforce and Parrot uses the same importation'''
    from transformers import AutoTokenizer, AutoModelForCausalLM
    tokenizer = AutoTokenizer.from_pretrained(model_type, cache_dir = cache_path )
    model = []
    logging.info("Tokenizer Loaded")
    if returnModel:
        model = AutoModelForCausalLM.from_pretrained(model_type, cache_dir = cache_path)
        logging.info("Model Loaded")        
    
    logging.info(model_type)
    return tokenizer, model
    
    

def init_model_args( current_case, returnModel = False ): 
    
    code_models = {
        'c1':('EleutherAI/gpt-neo-125m', ), # Basic (on Pile) GPT-3/J
        'c2':('EleutherAI/gpt-neo-1.3B', ),
        'c3':('EleutherAI/gpt-neo-2.7B', ),
        'c4':('EleutherAI/gpt-j-6b', ),
        'c5':('Salesforce/codegen-350M-nl', ), #Basic (on Pile) codegen
        'c6':('Salesforce/codegen-2B-nl', ),
        'c7':('Salesforce/codegen-6B-nl', ),
        'c8':('Salesforce/codegen-16B-nl', ),
        'c9':('codeparrot/codeparrot-small-multi', ), #multi-Language
        'c10':('Salesforce/codegen-350M-multi', ),
        'c11':('Salesforce/codegen-2B-multi', ),
        'c12':('Salesforce/codegen-6B-multi', ),
        'c13':('Salesforce/codegen-16B-multi', ),
        'c14':('codeparrot/codeparrot-small', ), #mono-Language
        'c15':('codeparrot/codeparrot', ),
        'c16':('Salesforce/codegen-350M-mono', ),
        'c17':('Salesforce/codegen-2B-mono', ),
        'c18':('Salesforce/codegen-6B-mono', ),
        'c19':('Salesforce/codegen-16B-mono', ),
    }
    
    model_type = code_models[current_case][0]
    tokenizer, model =  c_eleuther( returnModel = returnModel,  model_type = model_type ) 
    
    
    return model_type, tokenizer, model

# Init Parameters
> Loading Models and Testbeds


In [10]:
# [WARNING] Hyperparameters changes, please tune them up
params = {
    'codemodel' : 'c1',
    'numpy_files_logits_path': '../datax/1_numpy_files_logits/c1',
    'testbeds_path' : '../data/testbeds/AstEvalVerticalFiltered.json',
}

In [11]:
#Testing data loads
data_pd = pd.read_json( params['testbeds_path'] )
data_pd.head(1)

,size,ids,m_name,code,ast_errors,n_ast_errors,ast_levels,n_whitespaces_,complexity,nloc,token_counts,n_ast_nodes
0,280,"[4299, 1057, 62, 29412, 62, 41989, 7, 9288, 62...",c1,"def run_python_tests(test_modules, parallelism...",[],0,13,67,7,15,120,154


In [12]:
print(len(data_pd.index))

968449


In [13]:
data_pd.shape[0]

968449

In [14]:
#Uploading Model UnderAnalisys
name, tokenizer, model = init_model_args(
    current_case = params['codemodel'], 
    returnModel = True #[WARNING!] Check the parameters before calling it. 
    )

# Extracting Logits From a Given Model

### Creating data tensors

In [15]:
filtered_prompts_ids = data_pd[data_pd['m_name']==params['codemodel']]['ids'].values

In [16]:
filtered_prompts_ids.shape[0]

50971

In [18]:
#Casting Integers to Tensor Integers. Make sure the tesor is created in a device
#We ignored the parameter attention_mask since we are not using masking here [https://huggingface.co/transformers/v4.10.1/glossary.html#attention-mask]

tf_input_ids = [torch.tensor(  input_ids, dtype = torch.int, device=device ) for input_ids in filtered_prompts_ids ]

### Loading Model to Memory

In [19]:
model.to( device ) #WARNING, Verify the device before assigning to memory

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3

### Executing Logits

In [20]:
import os

def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [21]:
def logit_extractor(batch, input, from_index=0):
    """
    Output is the class CausalLMOutputWithPast (https://huggingface.co/transformers/v4.10.1/main_classes/output.html?highlight=causallmoutputwithpast)"
    logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size)) – Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
    The expression i.type(torch.LongTensor).to(device) is for casting labels for the loss
    """
    #Output is in CausalLMOutputWithPast
    CODEMODEL =  params['codemodel']
    create_folder(params['numpy_files_logits_path'])
    for idx, n in enumerate( range( from_index, len(input), batch) ):
        output = [ model( 
            input_ids = i, 
            labels = i.to(torch.long).to(device) 
            ) for i in input[n:n+batch] ] #Labels must be provided to compute loss
    
        output_logits = [ o.logits.detach().to('cpu').numpy() for o in output ]  #Logits Extraction
        output_loss = np.array([ o.loss.detach().to('cpu').numpy() for o in output ])  #Language modeling loss (for next-token prediction).

        #Saving Callbacks
        current_batch = idx + (from_index//batch)
        for jdx, o_logits in enumerate( output_logits ):
            np.save( params['numpy_files_logits_path']+ '/'+ f'logits_tensor[{jdx+n}]_batch[{current_batch}]_model[{CODEMODEL}].npy', o_logits) #Saving LOGITS
        np.save( params['numpy_files_logits_path']+ '/'+f'loss_batch[{current_batch}]_model[{CODEMODEL}].npy', output_loss) #Saving LOSS
        
        logging.info(f"Batch [{current_batch}] Completed")
        
        #Memory Released
        for out in output:
            del out.logits
            del out.loss
            del out
            torch.cuda.empty_cache()
            
        for out in output_logits:
            del out
            torch.cuda.empty_cache()
        for out in output_loss:
            del out
            torch.cuda.empty_cache()
        del output
        del output_logits
        del output_loss
    
    pass

In [22]:
## ACTUAL EXPERIMENT
## TIME AND MEMORY CONSUMING
logit_extractor(
    batch = 1, 
    input = tf_input_ids, 
    from_index=0
)

OSError: Not enough free space to write 38999432 bytes

In [ ]:
#logit_extractor(batch =2, input= input_ids_list[:2]) #<---- [WARNING TIME AND MEMORY CONSUMING]

In [ ]:
#output_logits = np.load('../data/callbacks/logits_tensor[0]_batch[0].npy')

In [ ]:
#assert output_logits.shape[0] == len(input_ids_list[0])

In [ ]:
#output_loss = np.load('../data/callbacks/loss_batch[0].npy')

In [ ]:
#output_loss